In [9]:
import time
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#### Trying to understand paths for book title, author and price

In [2]:
title_1 = "/html/body/section[3]/div/div/div[2]/div[1]/div[1]/div/div/h1/a"
author_1 = "/html/body/section[3]/div/div/div[2]/div[1]/div[1]/div/div/h2"
price_1 = "/html/body/section[3]/div/div/div[2]/div[1]/div[1]/div/div/div[4]/div"

title_2 = "/html/body/section[3]/div/div/div[2]/div[1]/div[2]/div/div/h1/a"
author_2 = "/html/body/section[3]/div/div/div[2]/div[1]/div[2]/div/div/h2"
price_2 = "/html/body/section[3]/div/div/div[2]/div[1]/div[2]/div/div/div[4]/div"

#Another page
title_3 = "/html/body/section[3]/div/div/div[2]/div[1]/div[1]/div/div/h1/a"
author_3 = "/html/body/section[3]/div/div/div[2]/div[1]/div[1]/div/div/h2"
price_3 = "/html/body/section[3]/div/div/div[2]/div[1]/div[1]/div/div/div[4]/div"

### For scraping data about author, title and price for one genre

In [8]:
# # If there is a problem, you can download webdriver from (https://chromedriver.chromium.org/) and change executable path
# # driver = webdriver.Chrome(executable_path='C:/Users/Anna/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')
driver = webdriver.Chrome()
driver.get('https://zangakbookstore.am/hay-zhamanakakits-ardzak')

# Defining base XPath for the books
base_xpath = '/html/body/section[3]/div/div/div[2]/div[1]/div[{}]/div/div/'

full = pd.DataFrame()
fullest = pd.DataFrame()

# Looping through pages
for page_number in range(1, 24):
    # Navigating to the next page
    if page_number > 1:
        driver.get(f'https://zangakbookstore.am/hay-zhamanakakits-ardzak?page={page_number}')

    # Extracting data for each book on the page
    book_number = 1
    while True:
        current_xpath = base_xpath.format(book_number)

        # Constructing XPaths for the current book
        title_xpath = current_xpath + 'h1/a'
        author_xpath = current_xpath + 'h2'
        price_xpath = current_xpath + 'div[4]/div'

        try:
            # Extracting data for the current book
            title = driver.find_element(By.XPATH, title_xpath).text
            author = driver.find_element(By.XPATH, author_xpath).text
            price = driver.find_element(By.XPATH, price_xpath).text

            # Creating DataFrame and concatenate
            data = {'Title': title, 'Author': author, 'Price': price}
            df = pd.DataFrame(data, index=[0])
            full = pd.concat([full, df], axis=0, ignore_index=True)

            book_number += 1
        except NoSuchElementException:
            break  # Breaking the loop when no more books are found on the page

fullest = pd.concat([fullest, full], axis=1)
fullest.to_excel('zangak_data.xlsx', index=False)

# Closing the browser
driver.quit()

#### Trying to understand paths for more info and description

In [7]:
# /html/body/div[4]/div[1]/div[3]/div[1]/div[1]/ul/li[1]/a
# /html/body/div[4]/div[1]/div[3]/div[1]/div[1]/ul/li[1]/a
# /html/body/div[4]/div[1]/div[3]/div[1]/div[1]/ul/li[1]/a
# /html/body/div[4]/div[1]/div[3]/div[1]/div[1]/ul/li[2]/a
# /html/body/div[4]/div[1]/div[3]/div[1]/div[1]/ul/li[1]/a
# /html/body/div[4]/div[1]/div[3]/div[1]/div[1]/ul/li[1]/a

#Right ones
# //*[@id="tab_details"]/div
# //*[@id="tab_aboutBook"]/div

### For scraping data about author, title, price, description and more info for one genre

In [6]:
# If there is a problem, you can download webdriver from (https://chromedriver.chromium.org/) and change executable path
# driver = webdriver.Chrome(executable_path='C:/Users/Anna/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')
driver.get('https://zangakbookstore.am/hay-zhamanakakits-ardzak')

# Defining base XPath for the books
base_xpath = '/html/body/section[3]/div/div/div[2]/div[1]/div[{}]/div/div/'

full = pd.DataFrame()
fullest = pd.DataFrame()

# Looping through pages
for page_number in range(1, 24):
    # Navigating to the next page
    if page_number > 1:
        driver.get(f'https://zangakbookstore.am/hay-zhamanakakits-ardzak?page={page_number}')

    # Extracting data for each book on the page
    book_number = 1
    while True:
        current_xpath = base_xpath.format(book_number)

        # Constructing XPaths for the current book
        title_xpath = current_xpath + 'h1/a'
        author_xpath = current_xpath + 'h2'
        price_xpath = current_xpath + 'div[4]/div'
        description_xpath = '//*[@id="tab_details"]/div'
        more_info_xpath = '//*[@id="tab_aboutBook"]/div'

        try:
            # Extracting data for the current book
            title = driver.find_element(By.XPATH, title_xpath).text
            author = driver.find_element(By.XPATH, author_xpath).text
            price = driver.find_element(By.XPATH, price_xpath).text

            # Scrolling the element into view
            element_to_click = driver.find_element(By.XPATH, title_xpath)
            driver.execute_script("arguments[0].scrollIntoView(true);", element_to_click)
            time.sleep(2)  # Add a delay to allow the scroll to complete

            # Clicking the element using JavaScript
            driver.execute_script("arguments[0].click();", element_to_click)

            # Waiting for the loader to disappear
            WebDriverWait(driver, 10).until_not(EC.presence_of_element_located((By.CLASS_NAME, 'page-loader')))
            
            # Extracting additional details
            try:
                description = driver.find_element(By.XPATH, description_xpath).text.replace('\n', '-')
            except NoSuchElementException:
                description = "NA"

            try:
                more_info = driver.find_element(By.XPATH, more_info_xpath).text.replace('\n', '-')
            except NoSuchElementException:
                more_info = "NA"

            # Creating DataFrame and concatenating
            data = {'Title': title, 'Author': author, 'Price': price, 'Description': description, 'More Info': more_info}
            df = pd.DataFrame(data, index=[0])
            full = pd.concat([full, df], axis=0, ignore_index=True)

            # Going back to the main page
            driver.execute_script("window.history.go(-1)")

            # Waiting for the loader to disappear after going back
            WebDriverWait(driver, 10).until_not(EC.presence_of_element_located((By.CLASS_NAME, 'page-loader')))

            book_number += 1
        except NoSuchElementException:
            break  # Breaking the loop when no more books are found on the page

fullest = pd.concat([fullest, full], axis=1)
fullest.to_excel('zangak_data_formatted.xlsx', index=False)

# Closing the browser
driver.quit()

### List of genre names with their links

- Fiction Books:

In [ ]:
# https://zangakbookstore.am/hay-zhamanakakits-ardzak
# https://zangakbookstore.am/hay-dasakan-grakanutyun
# https://zangakbookstore.am/artasahmanyan-zhamanakakits-ardzak
# https://zangakbookstore.am/artasahmanyan-dasakan-ardzak
# https://zangakbookstore.am/patmavep
# https://zangakbookstore.am/siravep
# https://zangakbookstore.am/fentzi
# https://zangakbookstore.am/gitakan-fantastika
# https://zangakbookstore.am/detektiv-ev-triller
# https://zangakbookstore.am/sarsap
# https://zangakbookstore.am/dramaturgia
# https://zangakbookstore.am/poezia
# https://zangakbookstore.am/epos-ev-banahyusutyun

In [22]:
urls_fiction = [
    "https://zangakbookstore.am/hay-zhamanakakits-ardzak",
    "https://zangakbookstore.am/hay-dasakan-grakanutyun",
    "https://zangakbookstore.am/artasahmanyan-zhamanakakits-ardzak",
    "https://zangakbookstore.am/artasahmanyan-dasakan-ardzak",
    "https://zangakbookstore.am/patmavep",
    "https://zangakbookstore.am/siravep",
    "https://zangakbookstore.am/fentzi",
    "https://zangakbookstore.am/gitakan-fantastika",
    "https://zangakbookstore.am/detektiv-ev-triller",
    "https://zangakbookstore.am/sarsap",
    "https://zangakbookstore.am/dramaturgia",
    "https://zangakbookstore.am/poezia",
    "https://zangakbookstore.am/epos-ev-banahyusutyun"
]

elements_1 = [url.split("https://zangakbookstore.am/")[-1] for url in urls_fiction]

print(elements_1)
#print(urls)

['hay-zhamanakakits-ardzak', 'hay-dasakan-grakanutyun', 'artasahmanyan-zhamanakakits-ardzak', 'artasahmanyan-dasakan-ardzak', 'patmavep', 'siravep', 'fentzi', 'gitakan-fantastika', 'detektiv-ev-triller', 'sarsap', 'dramaturgia', 'poezia', 'epos-ev-banahyusutyun']


In [1]:
# Checkpoint filename
checkpoint_file = 'scrapingzangak_checkpoint.pkl'

# List of URLs for fiction genres
urls_fiction = [
    "https://zangakbookstore.am/hay-zhamanakakits-ardzak", #24 pages
    "https://zangakbookstore.am/hay-dasakan-grakanutyun?languages=1", # 9 pages
    "https://zangakbookstore.am/artasahmanyan-zhamanakakits-ardzak?languages=1", # 161 pages 
    "https://zangakbookstore.am/artasahmanyan-dasakan-ardzak?languages=1",#173 pages
    "https://zangakbookstore.am/patmavep?languages=1", #9 pages
    "https://zangakbookstore.am/siravep?languages=1", #16 pages
    "https://zangakbookstore.am/fentzi?languages=1", #23 pages - didn't complete fully
    "https://zangakbookstore.am/gitakan-fantastika?languages=1", #30 pages - 
    "https://zangakbookstore.am/detektiv-ev-triller?languages=1", #44 pages
    "https://zangakbookstore.am/sarsap?languages=1", #7 pages
    "https://zangakbookstore.am/dramaturgia?languages=1", #10 pages
    "https://zangakbookstore.am/poezia?languages=1", #32 pages
    "https://zangakbookstore.am/epos-ev-banahyusutyun?languages=1" #4 pages
]

# Checking if checkpoint file exists
if os.path.exists(checkpoint_file):
    # Loading checkpoint
    fullest = pd.read_pickle(checkpoint_file)
else:
    fullest = pd.DataFrame()

# Initializing the WebDriver
driver = webdriver.Chrome(executable_path='C:/Users/Anna/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')

# Looping through each URL
for url in urls_fiction:
    driver.get(url)
    base_xpath = '/html/body/section[3]/div/div/div[2]/div[1]/div[{}]/div/div/'

    # Looping through pages
    for page_number in range(1, 30):
        # Navigate to the next page
        if page_number > 1:
            driver.get(f'{url}&page={page_number}')

        # Extracting data for each book on the page
        book_number = 1
        while True:
            current_xpath = base_xpath.format(book_number)

            # Constructing XPaths for the current book
            title_xpath = current_xpath + 'h1/a'
            author_xpath = current_xpath + 'h2'
            price_xpath = current_xpath + 'div[4]/div'
            description_xpath = '//*[@id="tab_details"]/div'
            more_info_xpath = '//*[@id="tab_aboutBook"]/div'

            try:
                # Extracting data for the current book
                title = driver.find_element(By.XPATH, title_xpath).text
                author = driver.find_element(By.XPATH, author_xpath).text
                price = driver.find_element(By.XPATH, price_xpath).text

                # Scrolling the element into view
                element_to_click = driver.find_element(By.XPATH, title_xpath)
                driver.execute_script("arguments[0].scrollIntoView(true);", element_to_click)
                time.sleep(2)  # Add a delay to allow the scroll to complete

                # Clicking the element using JavaScript
                driver.execute_script("arguments[0].click();", element_to_click)

                # Waiting for the loader to disappear
                WebDriverWait(driver, 10).until_not(EC.presence_of_element_located((By.CLASS_NAME, 'page-loader')))
                
                # Extracting additional details
                try:
                    description = driver.find_element(By.XPATH, description_xpath).text.replace('\n', '-')
                except NoSuchElementException:
                    description = "NA"

                try:
                    more_info = driver.find_element(By.XPATH, more_info_xpath).text.replace('\n', '-')
                except NoSuchElementException:
                    more_info = "NA"

                # Creating DataFrame and concatenate
                data = {'Title': title, 'Author': author, 'Price': price, 'Description': description, 'More Info': more_info, 'URL': url}
                df = pd.DataFrame(data, index=[0])
                fullest = pd.concat([fullest, df], axis=0, ignore_index=True)

                # Going back to the main page
                driver.execute_script("window.history.go(-1)")

                # Waiting for the loader to disappear after going back
                WebDriverWait(driver, 25).until(EC.invisibility_of_element_located((By.CLASS_NAME, 'page-loader')) 
                                                or EC.presence_of_element_located((By.CLASS_NAME, 'page-loader')))

                book_number += 1
            except NoSuchElementException:
                break  # Break the loop when no more books are found on the page

    # Saving the data for the current genre to an Excel file
    genre_name = url.split("/")[-1].split("?")[0]  # Extracting genre name from URL
    genre_filename = f'zangak_data_{genre_name}_books.xlsx'
    fullest.to_excel(genre_filename, index=False)
    print(f"Saved data for genre: {genre_name}")

    # Reseting 'fullest' DataFrame for the next genre
    fullest = pd.DataFrame()

# Saving the final data to an Excel file
fullest.to_excel('zangak_data_fiction_books.xlsx', index=False)

# Closing the browser
driver.quit()

Saved data for genre: siravep
Saved data for genre: fentzi
Saved data for genre: gitakan-fantastika
Saved data for genre: detektiv-ev-triller
Saved data for genre: sarsap
Saved data for genre: dramaturgia
Saved data for genre: poezia
Saved data for genre: epos-ev-banahyusutyun


- Non-fiction books

In [17]:
# https://zangakbookstore.am/arvest-ev-mshakuyt
# https://zangakbookstore.am/biznes-ev-tntesagitutyun
# https://zangakbookstore.am/bnakan-gitutyunner
# https://zangakbookstore.am/bzhshkakan-grakanutyun
# https://zangakbookstore.am/dizayn-ev-lusankarchutyun
# https://zangakbookstore.am/kino-ev-tatron
# https://zangakbookstore.am/erazhshtutyun
# https://zangakbookstore.am/ezoterika
# https://zangakbookstore.am/kensagrutyun-ev-husher
# https://zangakbookstore.am/noradzevutyun-ev-och
# https://zangakbookstore.am/inqnazargatsum-ev-motivatsia
# https://zangakbookstore.am/lezvabanutyun-ev-grakanagitutyun
# https://zangakbookstore.am/khohararutyun
# https://zangakbookstore.am/hogebanutyun
# https://zangakbookstore.am/tsnokhavarutyun
# https://zangakbookstore.am/kron
# https://zangakbookstore.am/hobbi
# https://zangakbookstore.am/hasarakakan-gitutyunner
# https://zangakbookstore.am/hraparakakhosutyun
# https://zangakbookstore.am/chanaparhordutyun
# https://zangakbookstore.am/chartarapetutyun
# https://zangakbookstore.am/patmutyun
# https://zangakbookstore.am/sport-ev-zhamants
# https://zangakbookstore.am/iravagitutyun
# https://zangakbookstore.am/tt-ev-tsragravorum
# https://zangakbookstore.am/pastavaveragrutyun
# https://zangakbookstore.am/pilisopayutyun
# https://zangakbookstore.am/qakhaqagitutyun
# https://zangakbookstore.am/amsagrer

In [23]:
urls_non_fiction = [
    "https://zangakbookstore.am/arvest-ev-mshakuyt",
    "https://zangakbookstore.am/biznes-ev-tntesagitutyun",
    "https://zangakbookstore.am/bnakan-gitutyunner",
    "https://zangakbookstore.am/bzhshkakan-grakanutyun",
    "https://zangakbookstore.am/dizayn-ev-lusankarchutyun",
    "https://zangakbookstore.am/kino-ev-tatron",
    "https://zangakbookstore.am/erazhshtutyun",
    "https://zangakbookstore.am/ezoterika",
    "https://zangakbookstore.am/kensagrutyun-ev-husher",
    "https://zangakbookstore.am/noradzevutyun-ev-och",
    "https://zangakbookstore.am/inqnazargatsum-ev-motivatsia",
    "https://zangakbookstore.am/lezvabanutyun-ev-grakanagitutyun",
    "https://zangakbookstore.am/khohararutyun",
    "https://zangakbookstore.am/hogebanutyun",
    "https://zangakbookstore.am/tsnokhavarutyun",
    "https://zangakbookstore.am/kron",
    "https://zangakbookstore.am/hobbi",
    "https://zangakbookstore.am/hasarakakan-gitutyunner",
    "https://zangakbookstore.am/hraparakakhosutyun",
    "https://zangakbookstore.am/chanaparhordutyun",
    "https://zangakbookstore.am/chartarapetutyun",
    "https://zangakbookstore.am/patmutyun",
    "https://zangakbookstore.am/sport-ev-zhamants",
    "https://zangakbookstore.am/iravagitutyun",
    "https://zangakbookstore.am/tt-ev-tsragravorum",
    "https://zangakbookstore.am/pastavaveragrutyun",
    "https://zangakbookstore.am/pilisopayutyun",
    "https://zangakbookstore.am/qakhaqagitutyun",
    "https://zangakbookstore.am/amsagrer"
]

elements_2 = [url.split("https://zangakbookstore.am/")[-1] for url in urls_non_fiction]

print(elements_2)
#print(urls_non_fiction)

['arvest-ev-mshakuyt', 'biznes-ev-tntesagitutyun', 'bnakan-gitutyunner', 'bzhshkakan-grakanutyun', 'dizayn-ev-lusankarchutyun', 'kino-ev-tatron', 'erazhshtutyun', 'ezoterika', 'kensagrutyun-ev-husher', 'noradzevutyun-ev-och', 'inqnazargatsum-ev-motivatsia', 'lezvabanutyun-ev-grakanagitutyun', 'khohararutyun', 'hogebanutyun', 'tsnokhavarutyun', 'kron', 'hobbi', 'hasarakakan-gitutyunner', 'hraparakakhosutyun', 'chanaparhordutyun', 'chartarapetutyun', 'patmutyun', 'sport-ev-zhamants', 'iravagitutyun', 'tt-ev-tsragravorum', 'pastavaveragrutyun', 'pilisopayutyun', 'qakhaqagitutyun', 'amsagrer']
